# Omega Analyses

In [1]:
# Import packages.
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import re
import scipy.stats as stats
# Intialize the matplolib styling.
plt.rcParams.update({
    'figure.constrained_layout.use': True,
    'figure.facecolor': 'white',
    'font.family': 'serif',
    'axes.spines.top': False,
    'axes.spines.right': False,
})
# Intialize pandas display options.
pd.set_option('display.max_columns', None)

## Coordinates for All 1:1 Orthologs

In [2]:
# Read in the san gff file.
san_gff_df = pd.read_csv(
    '../annotations/Dsan.genomic.gff.gz', sep='\t', skiprows=8, skipfooter=1,
    names=['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'],
)
# Read in the yak gff file.
yak_gff_df = pd.read_csv(
    '../annotations/Dyak.genomic.gff.gz', sep='\t', skiprows=8, skipfooter=1,
    names=['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'],
)
# Filter everything, but genes.
san_gene_gff_df = san_gff_df[(san_gff_df['type'] == 'gene') & (san_gff_df['source'] == 'Gnomon')].reset_index(drop=True)
yak_gene_gff_df = yak_gff_df[(yak_gff_df['type'] == 'gene') & (yak_gff_df['source'] == 'Gnomon')].reset_index(drop=True)

/tmp/ipykernel_21263/1818015581.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  san_gff_df = pd.read_csv(
/tmp/ipykernel_21263/1818015581.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  yak_gff_df = pd.read_csv(


In [3]:
# Load the 1:1 ortholog data.
ortholog_df = pd.read_csv(
    '../annotations/orthogroup_mel_san_yak_mapping.txt',
    sep='\t', names=['orthogroup', 'mel', 'san', 'yak'],
).sort_values('mel')
# Extract the orthogroup.
orthogroups = ortholog_df['orthogroup'].values
ortholog_df

,orthogroup,mel,san,yak
4718,OG0005672,128up,LOC120445982,LOC6530201
10332,OG0011289,14-3-3epsilon,LOC120453158,LOC6535530
4506,OG0005460,14-3-3zeta,LOC120450790,LOC6528869
9895,OG0010852,140up,LOC120453033,LOC6537351
7867,OG0008822,26-29-p,LOC120448918,LOC6534061
...,...,...,...,...
7103,OG0008057,zpg,LOC120448714,LOC6533216
3191,OG0004141,zuc,LOC120452839,LOC6527326
1870,OG0002819,zyd,LOC120456702,LOC6526241
8558,OG0009514,zye,LOC120448936,LOC26536128


In [4]:
# Load all of the genes of interest data frames.
obp_df = pd.read_csv(
    '../annotations/obp_orthogroups.txt',
    sep='\t', names=['orthogroup', 'mel', 'san', 'yak'],
)
or_df = pd.read_csv(
    '../annotations/or_orthogroups.txt',
    sep='\t', names=['orthogroup', 'mel', 'san', 'yak'],
)
gr_df = pd.read_csv(
    '../annotations/gr_orthogroups.txt',
    sep='\t', names=['orthogroup', 'mel', 'san', 'yak'],
)
ir_df = pd.read_csv(
    '../annotations/ir_orthogroups.txt',
    sep='\t', names=['orthogroup', 'mel', 'san', 'yak'],
)
# Create dictionaries per gene family.
obp_dicc = {key: 'Obp' for key in obp_df['orthogroup'].values}
or_dicc = {key: 'Or' for key in or_df['orthogroup'].values}
gr_dicc = {key: 'Gr' for key in gr_df['orthogroup'].values}
ir_dicc = {key: 'Ir' for key in ir_df['orthogroup'].values}
# Merge the dictionaries.
olfactory_dicc = {}
olfactory_dicc.update(obp_dicc)
olfactory_dicc.update(or_dicc)
olfactory_dicc.update(gr_dicc)
olfactory_dicc.update(ir_dicc)
# Determine which genes are in the genomic background.
genomic_background = np.setdiff1d(orthogroups, np.array(list(olfactory_dicc.keys())))
# Build a genomic background dictionary.
gb_dicc = {key: 'GB' for key in genomic_background}
# Construct the final dictionary.
gene_dicc = {}
gene_dicc.update(olfactory_dicc)
gene_dicc.update(gb_dicc)

In [5]:
# Load the paml results.
paml_df = pd.read_csv(
    './tables/paml_results_mel_san_yak_tei.csv',
).sort_values('gene_id')
paml_df

,gene_id,pw_Dmel_Dsan_w,pw_Dtei_Dmel_w,pw_Dtei_Dsan_w,pw_Dyak_Dmel_w,pw_Dyak_Dsan_w,pw_Dyak_Dtei_w,b_Dmel_brl,b_Dtei_brl,b_DsanDyak_brl,b_Dsan_brl,b_Dyak_brl,b_Dmel_w,b_Dtei_w,b_DsanDyak_w,b_Dsan_w,b_Dyak_w,M0_w01,M1_w01,M2_w01,M3_w01,M7_w01,M8_w01,M0_w1,M1_w1,M2_w1,M3_w1,M7_w1,M8_w1,M0_w2,M1_w2,M2_w2,M3_w2,M7_w2,M8_w2,BS_H0_Dmel,BS_H0_Dsan,BS_H0_Dtei,BS_H0_DsanDyak,BS_H0_Dyak,BS_H1_Dmel_w01,BS_H1_Dmel_w1,BS_H1_Dmel_w2,BS_H1_Dsan_w01,BS_H1_Dsan_w1,BS_H1_Dsan_w2,BS_H1_Dtei_w01,BS_H1_Dtei_w1,BS_H1_Dtei_w2,BS_H1_DsanDyak_w01,BS_H1_DsanDyak_w1,BS_H1_DsanDyak_w2,BS_H1_Dyak_w01,BS_H1_Dyak_w1,BS_H1_Dyak_w2,gene_tree,P_val_M0M3,P_val_M1M2,P_val_M7M8,P_val_Dmel,P_val_Dsan,P_val_Dtei,P_val_DsanDyak,P_val_Dyak,ps_scenario
0,128up,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.106,0.019,0.016,0.021,0.018,0.0001,0.0001,0.0001,0.0001,0.0001,-1729.916253,-1729.894184,-1729.892525,-1729.892513,-1729.892278,-1729.893949,-1729.916253,-1729.894184,-1729.892513,-1729.892513,-1729.892278,-1729.893949,-1729.916253,-1729.894184,-1729.892513,-1729.892513,-1729.892278,-1729.893949,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892517,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,-1729.892513,"(Dmel:0.0324899913,(Dsan:0.0065512672,Dyak:0.0...",8.275078e-01,0.953900,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000,No_PS
2,14-3-3epsilon,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.054,0.008,0.007,0.008,0.014,0.0001,0.0001,0.0001,0.0001,0.0001,-1139.562114,-1139.554848,-1139.554184,-1139.554183,-1139.554105,-1139.556877,-1139.562114,-1139.554848,-1139.554183,-1139.554183,-1139.554105,-1139.554800,-1139.562114,-1139.554848,-1139.554183,-1139.554183,-1139.554105,-1139.554769,-1139.554183,-1139.554183,-1139.554183,-1139.554183,-1139.554183,-1139.554183,-1139.554184,-1139.554186,-1139.554183,-1139.554183,-1139.554183,-1139.554185,-1139.554191,-1139.554183,-1139.554183,-1139.554183,-1139.554183,-1139.554183,-1139.554183,-1139.554183,"(Dmel:0.0549813304,(Dsan:0.0025443318,Dtei:0.0...",8.997759e-01,0.970908,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000,No_PS
3,14-3-3zeta,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.012,0.004,0.000,0.000,0.004,0.0001,0.0001,0.0001,999.0000,0.0001,-1011.537652,-1011.536836,-1016.714004,-1011.536747,-1011.536738,-1011.539110,-1011.537652,-1011.536836,-1016.714004,-1011.536747,-1011.536738,-1011.539110,-1011.537652,-1011.536836,-1011.536747,-1011.536747,-1011.536738,-1011.539290,-1011.536747,-1011.536747,-1011.536747,-1011.536747,-1011.536747,-1011.536747,-1011.536747,-1011.536747,-1011.536747,-1011.536749,-1011.536748,-1011.536747,-1011.536747,-1011.536747,-1011.536747,-1011.536748,-1011.536747,-1011.536747,-1011.536747,-1011.536748,"(Dmel:0.0040480685,(Dsan:0.0000010000,Dyak:0.0...",9.660650e-01,0.989355,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000,No_PS
1,140up,0.0876,0.1432,0.1092,0.0915,0.0010,0.1322,0.225,0.034,0.015,0.033,0.016,0.1111,0.4550,0.0001,0.0001,0.0001,-1408.810476,-1407.683327,-1407.683327,-1407.473212,-1407.520086,-1407.520099,-1408.810476,-1407.683327,-1407.683327,-1407.473212,-1407.520086,-1407.520099,-1408.810476,-1407.683327,-1407.683327,-1407.473212,-1407.520086,-1407.520099,-1406.478631,-1407.683327,-1403.671317,-1407.683327,-1407.683327,-1405.981644,-1407.683327,-1403.671317,-1407.683327,-1407.683327,-1405.981644,-1407.683327,-1403.671317,-1407.683327,-1407.683327,-1405.981644,-1407.683327,-1403.671317,-1407.683327,-1407.683327,"(Dmel:0.0685457702,(Dsan:0.0102027997,Dyak:0.0...",1.019656e-01,1.000000,1.000000,1.781116e-02,6.506352e-02,1.000000,6.506352e-02,0.004616,PS_unique_Dyak
4,26-29-p,0.0284,0.0341,0.0240,0.0262,0.0105,0.0215,0.117,0.051,0.025,0.025,0.023,0.0353,0.0293,0.0188,0.0193,0.0001,-2779.916221,-2779.916968,-2779.916221,-2779.916221,-2779.946134,-2779.946877,-2779.916221,-2779.916968,-2779.916267,-

In [6]:
# Extact the sorted mel ortholog data.
sorted_mel_genes = ortholog_df['mel'].values

In [7]:
# Extact the sorted gene ids from paml.
sorted_paml_genes = paml_df['gene_id'].values

In [8]:
# Define a function to extract gene ids.
def find_gene(x):
    # Define the pattern.
    pattern = re.compile(r'gene=([^;]*)')
    # Serach for the match.
    match = pattern.search(x)
    return match.group(1)

In [9]:
# Create a new column with the gene id information.
san_gene_gff_df['gene_id'] = san_gene_gff_df['attributes'].apply(find_gene)
yak_gene_gff_df['gene_id'] = yak_gene_gff_df['attributes'].apply(find_gene)

In [10]:
# Create seqid, start, and stop dictionaries.
san_contigs = {
    key: value for key, value in zip(san_gene_gff_df['gene_id'].values, san_gene_gff_df['seqid'].values)
}
san_starts = {
    key: value for key, value in zip(san_gene_gff_df['gene_id'].values, san_gene_gff_df['start'].values)
}
san_ends = {
    key: value for key, value in zip(san_gene_gff_df['gene_id'].values, san_gene_gff_df['end'].values)
}
yak_contigs = {
    key: value for key, value in zip(yak_gene_gff_df['gene_id'].values, yak_gene_gff_df['seqid'].values)
}
yak_starts = {
    key: value for key, value in zip(yak_gene_gff_df['gene_id'].values, yak_gene_gff_df['start'].values)
}
yak_ends = {
    key: value for key, value in zip(yak_gene_gff_df['gene_id'].values, yak_gene_gff_df['end'].values)
}

In [11]:
# Build the omega dataframe.
omega_df = pd.DataFrame({'orthogroup': orthogroups, 'gene_id': sorted_paml_genes})
# Add the gene type.
omega_df['gene_type'] = omega_df['orthogroup'].map(gene_dicc)

In [12]:
# Add the san gene ids, contigs, starts, and ends.
omega_df['san_id'] = ortholog_df['san'].values
omega_df['san_conting'] = omega_df['san_id'].map(san_contigs)
omega_df['san_start'] = omega_df['san_id'].map(san_starts)
omega_df['san_end'] = omega_df['san_id'].map(san_ends)

In [13]:
# Add the yak gene ids, contigs, starts, and ends.
omega_df['yak_id'] = ortholog_df['yak'].values
omega_df['yak_conting'] = omega_df['yak_id'].map(yak_contigs)
omega_df['yak_start'] = omega_df['yak_id'].map(yak_starts)
omega_df['yak_end'] = omega_df['yak_id'].map(yak_ends)

In [14]:
# Concatenate the meta data with the omega data.
pw_omega_df = pd.concat([omega_df, paml_df.iloc[:, 1:7]], axis=1)
new_paml_df = pd.concat([omega_df, paml_df.iloc[:, 1:]], axis=1)

In [15]:
# Export new files.
#pw_omega_df.to_csv('./tables/pw_omegas_mel_san_yak_tei_orthologs.txt', sep='\t', header=True, index=False)
#new_paml_df.to_csv('./tables/annotated_paml_results_mel_san_yak_tei.txt', sep='\t', header=True, index=False)

## PAML analyses

### Omega Analyses

In [16]:
# Intialize dictionaries.
pw_omega_dicc = {}
bl_omega_dicc = {}
# Intialize subdictionaries.
pw_omega_dicc['yak'] = {}
bl_omega_dicc['yak'] = {}
pw_omega_dicc['san'] = {}
bl_omega_dicc['san'] = {}
# For every gene type...
for gene in ['Obp', 'Or', 'Gr', 'Ir', 'GB']:
    # Extract the omega values.
    san_omega_pw = new_paml_df[new_paml_df['gene_type'] == gene]['pw_Dtei_Dsan_w'].values
    yak_omega_pw = new_paml_df[new_paml_df['gene_type'] == gene]['pw_Dyak_Dtei_w'].values
    san_omega_b = new_paml_df[new_paml_df['gene_type'] == gene]['b_Dsan_w'].values
    yak_omega_b = new_paml_df[new_paml_df['gene_type'] == gene]['b_Dyak_w'].values
    # Hardcode any site with no non-synon muts as 3.
    adj_san_omega_pw = np.where(san_omega_pw > 3, 3, san_omega_pw)
    adj_yak_omega_pw = np.where(yak_omega_pw > 3, 3, yak_omega_pw)
    adj_san_omega_b = np.where(san_omega_b > 3, 3, san_omega_b)
    adj_yak_omega_b = np.where(yak_omega_b > 3, 3, yak_omega_b)
    # Fill the dictionaries.
    pw_omega_dicc['san'][gene] = adj_san_omega_pw
    bl_omega_dicc['san'][gene] = adj_san_omega_b
    pw_omega_dicc['yak'][gene] = adj_yak_omega_pw
    bl_omega_dicc['yak'][gene] = adj_yak_omega_b

In [38]:
# Intialize a dictionaries to store the results.
pw_summary_dicc = {
    'N (san)': [], 'N (yak)': [],
    'AVG (san)': [], 'AVG (yak)': [],
    'STD (san)': [], 'STD (yak)': [],
}
bl_summary_dicc = {
    'N (san)': [], 'N (yak)': [],
    'AVG (san)': [], 'AVG (yak)': [],
    'STD (san)': [], 'STD (yak)': [],
}
# For every gene type...
for gene in ['Obp', 'Or', 'Gr', 'Ir', 'GB']:
    # Fill the pairwise dictionary.
    pw_summary_dicc['N (san)'].append(pw_omega_dicc['san'][gene].size)
    pw_summary_dicc['AVG (san)'].append(np.nanmean(pw_omega_dicc['san'][gene]))
    pw_summary_dicc['STD (san)'].append(np.nanstd(pw_omega_dicc['san'][gene]))
    pw_summary_dicc['N (yak)'].append(pw_omega_dicc['yak'][gene].size)
    pw_summary_dicc['AVG (yak)'].append(np.nanmean(pw_omega_dicc['yak'][gene]))
    pw_summary_dicc['STD (yak)'].append(np.nanstd(pw_omega_dicc['yak'][gene]))
    # Fill the branch dictionary.
    bl_summary_dicc['N (san)'].append(bl_omega_dicc['san'][gene].size)
    bl_summary_dicc['AVG (san)'].append(np.nanmean(bl_omega_dicc['san'][gene]))
    bl_summary_dicc['STD (san)'].append(np.nanstd(bl_omega_dicc['san'][gene]))
    bl_summary_dicc['N (yak)'].append(bl_omega_dicc['yak'][gene].size)
    bl_summary_dicc['AVG (yak)'].append(np.nanmean(bl_omega_dicc['yak'][gene]))
    bl_summary_dicc['STD (yak)'].append(np.nanstd(bl_omega_dicc['yak'][gene]))

In [39]:
# Show the pairwise omega summary.
pw_summary_df = pd.DataFrame(data=pw_summary_dicc, index=['Obp', 'Or', 'Gr', 'Ir', 'GB'])
pw_summary_df

,N (san),N (yak),AVG (san),AVG (yak),STD (san),STD (yak)
Obp,45,45,0.172271,0.222089,0.259519,0.469007
Or,45,45,0.161364,0.157222,0.139199,0.145275
Gr,50,50,0.137144,0.132628,0.131304,0.115045
Ir,48,48,0.124667,0.123035,0.130751,0.134296
GB,11655,11655,0.157626,0.154238,0.240138,0.236768


In [40]:
# Show the branch omega summary.
bl_summary_df = pd.DataFrame(data=bl_summary_dicc, index=['Obp', 'Or', 'Gr', 'Ir', 'GB'])
bl_summary_df

,N (san),N (yak),AVG (san),AVG (yak),STD (san),STD (yak)
Obp,45,45,0.267996,0.208264,0.612693,0.488313
Or,45,45,0.327253,0.195647,0.496206,0.260247
Gr,50,50,0.233010,0.269332,0.593580,0.696633
Ir,48,48,0.233573,0.143575,0.471929,0.175679
GB,11655,11655,0.281896,0.237505,0.569568,0.518544


In [43]:
# Intialize dictionaries to store the results.
pw_fet_dicc = {
    'OR (san)': [], 'P-val (san)': [],
    'OR (yak)': [], 'P-val (yak)': [],
}
bl_fet_dicc = {
    'OR (san)': [], 'P-val (san)': [],
    'OR (yak)': [], 'P-val (yak)': [],
}
# For every gene type...
for gene in ['Obp', 'Or', 'Gr', 'Ir']:
    # Perform a fisher's exact test for pairwise values.
    pw_san_or, pw_san_pval = stats.fisher_exact(
        [
            [(pw_omega_dicc['san'][gene] >= 1).sum(), pw_omega_dicc['san'][gene].size],
            [(pw_omega_dicc['san']['GB'] >= 1).sum(), pw_omega_dicc['san']['GB'].size],
        ]
    )
    pw_yak_or, pw_yak_pval = stats.fisher_exact(
        [
            [(pw_omega_dicc['yak'][gene] >= 1).sum(), pw_omega_dicc['yak'][gene].size],
            [(pw_omega_dicc['yak']['GB'] >= 1).sum(), pw_omega_dicc['yak']['GB'].size],
        ]
    )
    # Perform a fisher's exact test for branch values.
    bl_san_or, bl_san_pval = stats.fisher_exact(
        [
            [(bl_omega_dicc['san'][gene] >= 1).sum(), bl_omega_dicc['san'][gene].size],
            [(bl_omega_dicc['san']['GB'] >= 1).sum(), bl_omega_dicc['san']['GB'].size],
        ]
    )
    bl_yak_or, bl_yak_pval = stats.fisher_exact(
        [
            [(bl_omega_dicc['yak'][gene] >= 1).sum(), bl_omega_dicc['yak'][gene].size],
            [(bl_omega_dicc['yak']['GB'] >= 1).sum(), bl_omega_dicc['yak']['GB'].size],
        ]
    )
    # Fill the dictionaries.
    pw_fet_dicc['OR (san)'].append(pw_san_or)
    pw_fet_dicc['P-val (san)'].append(pw_san_pval)
    pw_fet_dicc['OR (yak)'].append(pw_yak_or)
    pw_fet_dicc['P-val (yak)'].append(pw_yak_pval)
    bl_fet_dicc['OR (san)'].append(bl_san_or)
    bl_fet_dicc['P-val (san)'].append(bl_san_pval)
    bl_fet_dicc['OR (yak)'].append(bl_yak_or)
    bl_fet_dicc['P-val (yak)'].append(bl_yak_pval)

In [44]:
# Show the pairwise omega fet.
pw_fet_df = pd.DataFrame(data=pw_fet_dicc, index=['Obp', 'Or', 'Gr', 'Ir'])
pw_fet_df

,OR (san),P-val (san),OR (yak),P-val (yak)
Obp,3.954198,0.097756,4.111111,0.091699
Or,0.000000,1.000000,0.000000,1.000000
Gr,0.000000,1.000000,0.000000,1.000000
Ir,0.000000,1.000000,0.000000,1.000000


In [45]:
# Show the pairwise omega fet.
bl_fet_df = pd.DataFrame(data=bl_fet_dicc, index=['Obp', 'Or', 'Gr', 'Ir'])
bl_fet_df

,OR (san),P-val (san),OR (yak),P-val (yak)
Obp,0.698113,1.000000,0.910369,1.000000
Or,0.698113,1.000000,0.455185,0.724084
Gr,0.942453,1.000000,1.228998,0.737109
Ir,0.654481,0.768481,0.000000,0.172178


In [50]:
# Intialize dictionaries to store the results.
pw_ttest_dicc = {
    'AVG (san)': [], 'AVG (yak)': [], 'P-val': [],
}
bl_ttest_dicc = {
    'AVG (san)': [], 'AVG (yak)': [], 'P-val': [],
}
# For every gene type...
for gene in ['Obp', 'Or', 'Gr', 'Ir', 'GB']:
    # Run a ttest on the pairwise results.
    pw_pval = stats.ttest_ind(
        pw_omega_dicc['san'][gene],
        pw_omega_dicc['yak'][gene],
        equal_var=False,
    )[1]
    # Run a ttest on the branch results.
    bl_pval = stats.ttest_ind(
        bl_omega_dicc['san'][gene],
        bl_omega_dicc['yak'][gene],
        equal_var=False,
    )[1]
    # Fill the dictionaries.
    pw_ttest_dicc['AVG (san)'].append(np.nanmean(pw_omega_dicc['san'][gene]))
    pw_ttest_dicc['AVG (yak)'].append(np.nanmean(pw_omega_dicc['yak'][gene]))
    pw_ttest_dicc['P-val'].append(pw_pval)
    bl_ttest_dicc['AVG (san)'].append(np.nanmean(bl_omega_dicc['san'][gene]))
    bl_ttest_dicc['AVG (yak)'].append(np.nanmean(bl_omega_dicc['yak'][gene]))
    bl_ttest_dicc['P-val'].append(bl_pval)

In [51]:
# Show the pairwise omega ttest.
pw_ttest_df = pd.DataFrame(data=pw_ttest_dicc, index=['Obp', 'Or', 'Gr', 'Ir', 'GB'])
pw_ttest_df

,AVG (san),AVG (yak),P-val
Obp,0.172271,0.222089,0.539608
Or,0.161364,0.157222,0.891689
Gr,0.137144,0.132628,0.856685
Ir,0.124667,0.123035,0.952549
GB,0.157626,0.154238,0.278033


In [52]:
# Show the pairwise omega ttest.
bl_ttest_df = pd.DataFrame(data=bl_ttest_dicc, index=['Obp', 'Or', 'Gr', 'Ir', 'GB'])
bl_ttest_df

,AVG (san),AVG (yak),P-val
Obp,0.267996,0.208264,6.143898e-01
Or,0.327253,0.195647,1.239733e-01
Gr,0.233010,0.269332,7.817620e-01
Ir,0.233573,0.143575,2.252880e-01
GB,0.281896,0.237505,5.005861e-10


### Testing Branches for Positive Selection

In [54]:
# Intialize a dictionary to store the results.
ps_branch_dicc = {}
# For every gene type...
for gene in ['Obp', 'Or', 'Gr', 'Ir', 'GB']:
    # Extract the positive selection scenario.
    ps_branch_dicc[gene] = new_paml_df[new_paml_df['gene_type'] == gene]['ps_scenario'].values

In [60]:
# Intialize a dictionary.
sce_freq_dicc = {
    'Total': []
}
# Intialize all possible scenarios list.
scenarios = [
    'No_PS', 'PS_ancestral', 'PS_independent',
    'PS_spill', 'PS_spill_Dsan', 'PS_spill_Dyak',
    'PS_unique_Dsan', 'PS_unique_Dyak',
]
# For every scenario...
for sce in scenarios:
    # Intialize the dictionary.
    sce_freq_dicc[sce] = []
# For every gene type...
for gene in ['Obp', 'Or', 'Gr', 'Ir', 'GB']:
    # For every scenario...
    for sce in scenarios:
        # Fill the dictionary
        sce_freq_dicc[sce].append(np.where(ps_branch_dicc[gene] == sce)[0].size / ps_branch_dicc[gene].size)
# For every gene type...
for gene in ['Obp', 'Or', 'Gr', 'Ir', 'GB']:
    # Fill the totals.
    sce_freq_dicc['Total'].append(ps_branch_dicc[gene].size)

In [61]:
# Show the positive selection branch results.
ps_branch_df = pd.DataFrame(data=sce_freq_dicc, index=['Obp', 'Or', 'Gr', 'Ir', 'GB'])
ps_branch_df

,Total,No_PS,PS_ancestral,PS_independent,PS_spill,PS_spill_Dsan,PS_spill_Dyak,PS_unique_Dsan,PS_unique_Dyak
Obp,45,0.400000,0.133333,0.022222,0.066667,0.244444,0.022222,0.022222,0.088889
Or,45,0.266667,0.088889,0.066667,0.111111,0.222222,0.044444,0.022222,0.177778
Gr,50,0.460000,0.100000,0.000000,0.140000,0.160000,0.000000,0.000000,0.140000
Ir,48,0.395833,0.062500,0.062500,0.062500,0.250000,0.062500,0.000000,0.104167
GB,11655,0.479451,0.065294,0.058344,0.093436,0.203260,0.015358,0.009953,0.074903


In [17]:
new_paml_df[new_paml_df['gene_id'] == 'Or24a']

,orthogroup,gene_id,gene_type,san_id,san_conting,san_start,san_end,yak_id,yak_conting,yak_start,yak_end,pw_Dmel_Dsan_w,pw_Dtei_Dmel_w,pw_Dtei_Dsan_w,pw_Dyak_Dmel_w,pw_Dyak_Dsan_w,pw_Dyak_Dtei_w,b_Dmel_brl,b_Dtei_brl,b_DsanDyak_brl,b_Dsan_brl,b_Dyak_brl,b_Dmel_w,b_Dtei_w,b_DsanDyak_w,b_Dsan_w,b_Dyak_w,M0_w01,M1_w01,M2_w01,M3_w01,M7_w01,M8_w01,M0_w1,M1_w1,M2_w1,M3_w1,M7_w1,M8_w1,M0_w2,M1_w2,M2_w2,M3_w2,M7_w2,M8_w2,BS_H0_Dmel,BS_H0_Dsan,BS_H0_Dtei,BS_H0_DsanDyak,BS_H0_Dyak,BS_H1_Dmel_w01,BS_H1_Dmel_w1,BS_H1_Dmel_w2,BS_H1_Dsan_w01,BS_H1_Dsan_w1,BS_H1_Dsan_w2,BS_H1_Dtei_w01,BS_H1_Dtei_w1,BS_H1_Dtei_w2,BS_H1_DsanDyak_w01,BS_H1_DsanDyak_w1,BS_H1_DsanDyak_w2,BS_H1_Dyak_w01,BS_H1_Dyak_w1,BS_H1_Dyak_w2,gene_tree,P_val_M0M3,P_val_M1M2,P_val_M7M8,P_val_Dmel,P_val_Dsan,P_val_Dtei,P_val_DsanDyak,P_val_Dyak,ps_scenario
8009,OG0003269,Or24a,Or,LOC120453978,NC_053016.2,4193748.0,4195216.0,LOC6526797,NC_052527.2,4271595.0,4273024.0,0.1591,0.1245,0.0377,0.1272,0.0453,0.0375,0.33,0.064,0.024,0.014,0.043,0.1604,0.0302,0.0687,0.0908,0.0334,-2708.238053,-2670.904487,-2662.276134,-2661.896012,-2671.890476,-2662.80726,-2708.238053,-2670.904487,-2662.275915,-2661.896585,-2671.890476,-2662.80744,-2708.238053,-2670.904487,-2662.27626,-2661.895861,-2671.890476,-2662.807098,-2659.967381,-2670.904487,-2670.904487,-2670.904487,-2670.904488,-2654.798267,-2670.904487,-2670.904487,-2670.904487,-2670.904487,-2654.798267,-2670.904487,-2670.904487,-2670.904487,-2670.904487,-2654.798267,-2669.328495,-2670.904487,-2670.904487,-2670.904488,"(Dmel:0.1310881874,(Dsan:0.0055154290,Dyak:0.0...",6.130000e-22,0.000033,0.00002,0.001303,1.380000e-08,1.0,1.380000e-08,0.998872,PS_spill_Dsan


In [18]:
new_paml_df[new_paml_df['gene_id'] == 'Or43b']

,orthogroup,gene_id,gene_type,san_id,san_conting,san_start,san_end,yak_id,yak_conting,yak_start,yak_end,pw_Dmel_Dsan_w,pw_Dtei_Dmel_w,pw_Dtei_Dsan_w,pw_Dyak_Dmel_w,pw_Dyak_Dsan_w,pw_Dyak_Dtei_w,b_Dmel_brl,b_Dtei_brl,b_DsanDyak_brl,b_Dsan_brl,b_Dyak_brl,b_Dmel_w,b_Dtei_w,b_DsanDyak_w,b_Dsan_w,b_Dyak_w,M0_w01,M1_w01,M2_w01,M3_w01,M7_w01,M8_w01,M0_w1,M1_w1,M2_w1,M3_w1,M7_w1,M8_w1,M0_w2,M1_w2,M2_w2,M3_w2,M7_w2,M8_w2,BS_H0_Dmel,BS_H0_Dsan,BS_H0_Dtei,BS_H0_DsanDyak,BS_H0_Dyak,BS_H1_Dmel_w01,BS_H1_Dmel_w1,BS_H1_Dmel_w2,BS_H1_Dsan_w01,BS_H1_Dsan_w1,BS_H1_Dsan_w2,BS_H1_Dtei_w01,BS_H1_Dtei_w1,BS_H1_Dtei_w2,BS_H1_DsanDyak_w01,BS_H1_DsanDyak_w1,BS_H1_DsanDyak_w2,BS_H1_Dyak_w01,BS_H1_Dyak_w1,BS_H1_Dyak_w2,gene_tree,P_val_M0M3,P_val_M1M2,P_val_M7M8,P_val_Dmel,P_val_Dsan,P_val_Dtei,P_val_DsanDyak,P_val_Dyak,ps_scenario
8018,OG0005168,Or43b,Or,LOC120458133,NC_053016.2,16669898.0,16671674.0,LOC6528495,NC_052527.2,16611415.0,16613040.0,0.0373,0.0475,0.0445,0.0377,0.0242,0.0361,0.181,0.06,0.032,0.031,0.019,0.046,0.054,0.0225,0.0372,0.0001,-4770.491681,-4764.646949,-4764.646949,-4764.629115,-4764.637346,-4764.637347,-4770.491681,-4764.646949,-4764.646949,-4764.629115,-4764.637346,-4764.637347,-4770.491681,-4764.646949,-4764.646949,-4764.629115,-4764.637346,-4764.637347,-4763.635839,-4764.646949,-4764.17526,-4764.646949,-4764.646949,-4762.83538,-4764.646949,-4764.17526,-4764.646949,-4764.646949,-4762.83538,-4764.646949,-4764.17526,-4764.646949,-4764.646949,-4762.83538,-4764.646949,-4764.17526,-4764.646949,-4764.646949,"(Dmel:0.0625054287,(Dsan:0.0100965166,Dyak:0.0...",0.000617,1.0,1.0,0.205773,0.056981,1.0,0.056981,0.331411,No_PS


In [19]:
new_paml_df[new_paml_df['gene_id'] == 'Ir93a']

,orthogroup,gene_id,gene_type,san_id,san_conting,san_start,san_end,yak_id,yak_conting,yak_start,yak_end,pw_Dmel_Dsan_w,pw_Dtei_Dmel_w,pw_Dtei_Dsan_w,pw_Dyak_Dmel_w,pw_Dyak_Dsan_w,pw_Dyak_Dtei_w,b_Dmel_brl,b_Dtei_brl,b_DsanDyak_brl,b_Dsan_brl,b_Dyak_brl,b_Dmel_w,b_Dtei_w,b_DsanDyak_w,b_Dsan_w,b_Dyak_w,M0_w01,M1_w01,M2_w01,M3_w01,M7_w01,M8_w01,M0_w1,M1_w1,M2_w1,M3_w1,M7_w1,M8_w1,M0_w2,M1_w2,M2_w2,M3_w2,M7_w2,M8_w2,BS_H0_Dmel,BS_H0_Dsan,BS_H0_Dtei,BS_H0_DsanDyak,BS_H0_Dyak,BS_H1_Dmel_w01,BS_H1_Dmel_w1,BS_H1_Dmel_w2,BS_H1_Dsan_w01,BS_H1_Dsan_w1,BS_H1_Dsan_w2,BS_H1_Dtei_w01,BS_H1_Dtei_w1,BS_H1_Dtei_w2,BS_H1_DsanDyak_w01,BS_H1_DsanDyak_w1,BS_H1_DsanDyak_w2,BS_H1_Dyak_w01,BS_H1_Dyak_w1,BS_H1_Dyak_w2,gene_tree,P_val_M0M3,P_val_M1M2,P_val_M7M8,P_val_Dmel,P_val_Dsan,P_val_Dtei,P_val_DsanDyak,P_val_Dyak,ps_scenario
7273,OG0011537,Ir93a,Ir,LOC120452183,NC_053019.2,5565567.0,5569268.0,LOC6535859,NC_052530.2,7354072.0,7358067.0,0.2594,0.2799,0.1259,0.2527,0.0585,0.1524,0.365,0.047,0.066,0.017,0.018,0.3257,0.1655,0.1101,0.0001,0.114,-1460.67363,-1456.413125,-1456.207055,-1456.207055,-1456.548322,-1456.20788,-1460.67363,-1456.413125,-1456.207055,-1456.207055,-1456.548322,-1456.20788,-1460.67363,-1456.413125,-1456.207055,-1456.207055,-1456.548322,-1456.20788,-1453.650388,-1456.413125,-1456.41019,-1456.413125,-1456.413125,-1453.650388,-1456.413125,-1456.41019,-1456.413125,-1456.413125,-1453.650388,-1456.413125,-1456.410191,-1456.413334,-1456.413125,-1453.650388,-1456.413125,-1456.41019,-1456.413125,-1456.413125,"(Dmel:0.1134703008,(Dsan:0.0054452180,Dyak:0.0...",0.0028,0.520885,0.409282,1.0,0.018742,1.0,0.018742,0.938929,PS_spill_Dsan


In [20]:
new_paml_df[new_paml_df['gene_id'] == 'Gr43a']

,orthogroup,gene_id,gene_type,san_id,san_conting,san_start,san_end,yak_id,yak_conting,yak_start,yak_end,pw_Dmel_Dsan_w,pw_Dtei_Dmel_w,pw_Dtei_Dsan_w,pw_Dyak_Dmel_w,pw_Dyak_Dsan_w,pw_Dyak_Dtei_w,b_Dmel_brl,b_Dtei_brl,b_DsanDyak_brl,b_Dsan_brl,b_Dyak_brl,b_Dmel_w,b_Dtei_w,b_DsanDyak_w,b_Dsan_w,b_Dyak_w,M0_w01,M1_w01,M2_w01,M3_w01,M7_w01,M8_w01,M0_w1,M1_w1,M2_w1,M3_w1,M7_w1,M8_w1,M0_w2,M1_w2,M2_w2,M3_w2,M7_w2,M8_w2,BS_H0_Dmel,BS_H0_Dsan,BS_H0_Dtei,BS_H0_DsanDyak,BS_H0_Dyak,BS_H1_Dmel_w01,BS_H1_Dmel_w1,BS_H1_Dmel_w2,BS_H1_Dsan_w01,BS_H1_Dsan_w1,BS_H1_Dsan_w2,BS_H1_Dtei_w01,BS_H1_Dtei_w1,BS_H1_Dtei_w2,BS_H1_DsanDyak_w01,BS_H1_DsanDyak_w1,BS_H1_DsanDyak_w2,BS_H1_Dyak_w01,BS_H1_Dyak_w1,BS_H1_Dyak_w2,gene_tree,P_val_M0M3,P_val_M1M2,P_val_M7M8,P_val_Dmel,P_val_Dsan,P_val_Dtei,P_val_DsanDyak,P_val_Dyak,ps_scenario
6949,OG0005094,Gr43a,Gr,LOC120450059,NC_053016.2,16161210.0,16163628.0,LOC6528411,NC_052527.2,16098523.0,16101167.0,0.0893,0.0837,0.0434,0.0952,0.001,0.0487,0.114,0.022,0.056,0.012,0.011,0.1095,0.0001,0.0834,0.0001,0.0001,-2227.627002,-2225.154001,-2225.15358,-2225.15358,-2225.163377,-2225.1538,-2227.627002,-2225.154001,-2225.15358,-2225.15358,-2225.163377,-2225.1538,-2227.627002,-2225.154001,-2225.154001,-2225.15358,-2225.163377,-2225.1538,-2222.286577,-2225.154043,-2225.154014,-2224.774912,-2225.154001,-2220.50256,-2225.154001,-2225.154001,-2224.774912,-2225.154001,-2220.50256,-2225.154001,-2225.154001,-2224.774912,-2225.154001,-2220.50256,-2225.154001,-2225.154001,-2224.774912,-2225.154001,"(Dmel:0.0356537913,(Dsan:0.0037948277,Dyak:0.0...",0.026138,0.976851,0.889926,0.058902,0.002288,0.383891,0.003465,0.383899,PS_spill_Dsan


In [21]:
new_paml_df[new_paml_df['gene_id'] == 'Gr77a']

,orthogroup,gene_id,gene_type,san_id,san_conting,san_start,san_end,yak_id,yak_conting,yak_start,yak_end,pw_Dmel_Dsan_w,pw_Dtei_Dmel_w,pw_Dtei_Dsan_w,pw_Dyak_Dmel_w,pw_Dyak_Dsan_w,pw_Dyak_Dtei_w,b_Dmel_brl,b_Dtei_brl,b_DsanDyak_brl,b_Dsan_brl,b_Dyak_brl,b_Dmel_w,b_Dtei_w,b_DsanDyak_w,b_Dsan_w,b_Dyak_w,M0_w01,M1_w01,M2_w01,M3_w01,M7_w01,M8_w01,M0_w1,M1_w1,M2_w1,M3_w1,M7_w1,M8_w1,M0_w2,M1_w2,M2_w2,M3_w2,M7_w2,M8_w2,BS_H0_Dmel,BS_H0_Dsan,BS_H0_Dtei,BS_H0_DsanDyak,BS_H0_Dyak,BS_H1_Dmel_w01,BS_H1_Dmel_w1,BS_H1_Dmel_w2,BS_H1_Dsan_w01,BS_H1_Dsan_w1,BS_H1_Dsan_w2,BS_H1_Dtei_w01,BS_H1_Dtei_w1,BS_H1_Dtei_w2,BS_H1_DsanDyak_w01,BS_H1_DsanDyak_w1,BS_H1_DsanDyak_w2,BS_H1_Dyak_w01,BS_H1_Dyak_w1,BS_H1_Dyak_w2,gene_tree,P_val_M0M3,P_val_M1M2,P_val_M7M8,P_val_Dmel,P_val_Dsan,P_val_Dtei,P_val_DsanDyak,P_val_Dyak,ps_scenario
6969,OG0009549,Gr77a,Gr,LOC120449167,NC_053018.2,19878544.0,19880441.0,LOC26535958,NC_052529.2,19860878.0,19865839.0,0.2026,0.247,0.2372,0.207,0.1554,0.2414,0.083,0.097,0.048,0.016,0.017,0.1992,0.2904,0.1889,0.1365,0.2423,-2224.864662,-2215.205591,-2214.547456,-2214.547456,-2215.310403,-2214.548559,-2224.864662,-2215.205591,-2214.547456,-2214.547456,-2215.310403,-2214.548559,-2224.864662,-2215.205591,-2214.547456,-2214.547456,-2215.310403,-2214.548559,-2215.205591,-2215.166387,-2215.205608,-2215.205422,-2215.173225,-2215.205591,-2212.923309,-2215.205591,-2215.205421,-2215.173225,-2215.205591,-2212.923274,-2215.205591,-2215.205413,-2215.173225,-2215.113523,-2212.622491,-2214.969789,-2215.205417,-2215.205591,"(Dmel:0.0287549442,(Dsan:0.0052143776,Dyak:0.0...",0.000006,0.251263,0.217062,0.03264,1.0,0.032638,0.023035,0.523561,PS_ancestral


In [22]:
new_paml_df[new_paml_df['gene_id'] == 'Or92a']

,orthogroup,gene_id,gene_type,san_id,san_conting,san_start,san_end,yak_id,yak_conting,yak_start,yak_end,pw_Dmel_Dsan_w,pw_Dtei_Dmel_w,pw_Dtei_Dsan_w,pw_Dyak_Dmel_w,pw_Dyak_Dsan_w,pw_Dyak_Dtei_w,b_Dmel_brl,b_Dtei_brl,b_DsanDyak_brl,b_Dsan_brl,b_Dyak_brl,b_Dmel_w,b_Dtei_w,b_DsanDyak_w,b_Dsan_w,b_Dyak_w,M0_w01,M1_w01,M2_w01,M3_w01,M7_w01,M8_w01,M0_w1,M1_w1,M2_w1,M3_w1,M7_w1,M8_w1,M0_w2,M1_w2,M2_w2,M3_w2,M7_w2,M8_w2,BS_H0_Dmel,BS_H0_Dsan,BS_H0_Dtei,BS_H0_DsanDyak,BS_H0_Dyak,BS_H1_Dmel_w01,BS_H1_Dmel_w1,BS_H1_Dmel_w2,BS_H1_Dsan_w01,BS_H1_Dsan_w1,BS_H1_Dsan_w2,BS_H1_Dtei_w01,BS_H1_Dtei_w1,BS_H1_Dtei_w2,BS_H1_DsanDyak_w01,BS_H1_DsanDyak_w1,BS_H1_DsanDyak_w2,BS_H1_Dyak_w01,BS_H1_Dyak_w1,BS_H1_Dyak_w2,gene_tree,P_val_M0M3,P_val_M1M2,P_val_M7M8,P_val_Dmel,P_val_Dsan,P_val_Dtei,P_val_DsanDyak,P_val_Dyak,ps_scenario
8044,OG0011489,Or92a,Or,LOC120452249,NC_053019.2,5208645.0,5210126.0,LOC6539949,NC_052530.2,6994695.0,6997094.0,0.0849,0.1012,0.0523,0.1149,0.1187,0.1151,0.188,0.036,0.033,0.023,0.015,0.1016,0.0882,0.052,0.0216,0.7259,-3379.184689,-3377.934934,-3377.934934,-3377.621493,-3377.67941,-3377.679447,-3379.184689,-3377.934934,-3377.934934,-3377.621493,-3377.67941,-3377.679447,-3379.184689,-3377.934934,-3377.934934,-3377.621493,-3377.67941,-3377.679447,-3377.934934,-3377.934934,-3377.934934,-3377.934934,-3375.261084,-3377.934934,-3377.934934,-3377.934934,-3377.934934,-3375.237675,-3377.934934,-3377.934934,-3377.934934,-3377.934934,-3375.237675,-3377.934934,-3377.934934,-3377.934934,-3377.934934,-3375.237675,"(Dmel:0.0642567219,(Dsan:0.0079923465,Dyak:0.0...",0.077034,1.0,1.0,1.0,0.0202,1.0,0.0202,0.828696,PS_spill_Dsan
